In [ ]:
library(seurat)

In [ ]:
# Import datasobj <- readRDS('{PATH_1}')

In [ ]:
colnames(sobj@meta.data)

[1] "nCount_ATAC"                 "nFeature_ATAC"              
 [3] "nCount_RNA"                  "nFeature_RNA"               
 [5] "orig.ident"                  "percent.mt"                 
 [7] "nucleosome_signal"           "nucleosome_percentile"      
 [9] "TSS.enrichment"              "TSS.percentile"             
[11] "nCount_SCT"                  "nFeature_SCT"               
[13] "predicted_doublets"          "doublet_scores"             
[15] "ATAC_snn_res.0.8"            "seurat_clusters"            
[17] "SCT.weight"                  "ATAC.weight"                
[19] "wsnn_res.0.8"                "guidecapture_classification"
[21] "S.Score"                     "G2M.Score"                  
[23] "Phase"                       "old.ident"                  
[25] "SCT_snn_res.0.8"             "Perturbation"               
[27] "isNT"                        "mixscape_class"             
[29] "mixscape_class_p_kd"         "mixscape_class.global"      
[31] "sample"                      "treatment"                  
[33] "treatment_and_guidecapture"  "nCount_PRTB"                
[35] "nFeature_PRTB"

In [15]:
Idents(sobj) <- sobj$treatment
table(Idents(sobj))


DASA DMSO 
4519 3475 

In [16]:
dasa_sobj <- subset(sobj, idents = "DASA")
dmso_sobj <- subset(sobj, idents = "DMSO")

In [18]:
Idents(dasa_sobj) <- dasa_sobj$treatment_and_guidecapture
Idents(dmso_sobj) <- dmso_sobj$treatment_and_guidecapture

# Create new col with just guide name (remove "DASA_" or "DMSO_")
dasa_sobj$guide <- gsub("DASA_", "", Idents(dasa_sobj))
dmso_sobj$guide <- gsub("DMSO_", "", Idents(dmso_sobj))


Idents(dasa_sobj) <- dasa_sobj$guide
Idents(dmso_sobj) <- dmso_sobj$guide


# Print table of idents
table(Idents(dasa_sobj))
table(Idents(dmso_sobj))



     NT    HIC2   PIAS1    CHD2 GPBP1L1   PQBP1   KMT2B    SLTM  PLAGL2  YEATS4 
   2615     375     309     226     131     102     210     221      83     121 
   ADNP    BRD2  ZNF669 
     15      32      79 


  KMT2B    CHD2      NT    BRD2 GPBP1L1    HIC2   PIAS1   ZBED6  YEATS4  ZNF669 
    152      51    1661     148     222     216     299     126     241     121 
   ADNP    SLTM     MNT 
     98     113      27 

In [22]:
calculate_perturb_knockdown <- function(seurat_obj, control_group = "NT") {
	# Set assay to SCT
	DefaultAssay(seurat_obj) <- "SCT"
	
	# Get all unique perturbations (excluding NT)
	perturbations <- setdiff(unique(Idents(seurat_obj)), control_group)
	
	# Initialize results list
	all_results <- list()
	
	# For each perturbation
	for(perturb in perturbations) {
		# Get cells for this condition
		control_cells <- WhichCells(seurat_obj, idents = control_group)
		perturb_cells <- WhichCells(seurat_obj, idents = perturb)
		
		# Calculate mean expression of the perturbed gene
		control_mean <- mean(GetAssayData(seurat_obj)[perturb, control_cells])
		perturb_mean <- mean(GetAssayData(seurat_obj)[perturb, perturb_cells])
		
		# Calculate knockdown percentage
		percent_kd <- (1 - (perturb_mean / control_mean)) * 100
		
		# Store results
		all_results[[perturb]] <- data.frame(
			target_gene = perturb,
			control_expression = control_mean,
			perturb_expression = perturb_mean,
			percent_knockdown = percent_kd
		)
	}
	
	# Combine all results
	final_results <- do.call(rbind, all_results)
	rownames(final_results) <- NULL
	
	return(final_results)
}


In [33]:
# Run for DASA
dasa_knockdown_results <- calculate_perturb_knockdown(dasa_sobj)
cat("DASA knockdown results:")
dasa_knockdown_results[order(dasa_knockdown_results$percent_knockdown, decreasing=TRUE),]

# Run for DMSO
dmso_knockdown_results <- calculate_perturb_knockdown(dmso_sobj)
cat("DMSO knockdown results:")
dmso_knockdown_results[order(dmso_knockdown_results$percent_knockdown, decreasing=TRUE),]

DASA knockdown results:

,target_gene,control_expression,perturb_expression,percent_knockdown
,<chr>,<dbl>,<dbl>,<dbl>
6,KMT2B,0.13337914,0.00000000,100.00000
7,SLTM,0.47773696,0.02248784,95.29284
4,GPBP1L1,0.76649898,0.04542475,94.07374
2,PIAS1,0.89817390,0.05421785,93.96355
10,ADNP,0.41333198,0.04620981,88.82017
3,CHD2,0.88299342,0.13292431,84.94617
9,YEATS4,0.12646318,0.02053642,83.76095
1,HIC2,0.48954566,0.12788931,73.87592
8,PLAGL2,0.11242858,0.04175585,62.86011


DMSO knockdown results:

,target_gene,control_expression,perturb_expression,percent_knockdown
,<chr>,<dbl>,<dbl>,<dbl>
2,CHD2,0.93074975,0.040773364,95.61930
6,PIAS1,0.90453822,0.051394719,94.31813
1,KMT2B,0.10579960,0.009120358,91.37959
4,GPBP1L1,0.66362239,0.067924950,89.76452
3,BRD2,0.30489862,0.032783988,89.24758
10,ADNP,0.36137351,0.074866703,79.28274
8,YEATS4,0.26678049,0.056328886,78.88568
11,SLTM,0.57856967,0.132403150,77.11544
7,ZBED6,0.02486523,0.011002336,55.75213


In [30]:
# Find overlapping genes
overlapping_genes <- intersect(dmso_knockdown_results$target_gene, dasa_knockdown_results$target_gene)

# Calculate mean knockdown
mean_knockdown <- data.frame(
	target_gene = overlapping_genes,
	dmso_kd = dmso_knockdown_results$percent_knockdown[match(overlapping_genes, dmso_knockdown_results$target_gene)],
	dasa_kd = dasa_knockdown_results$percent_knockdown[match(overlapping_genes, dasa_knockdown_results$target_gene)],
	mean_kd = NA  # We'll fill this next
)

# Calculate mean knockdown percentage
mean_knockdown$mean_kd <- rowMeans(mean_knockdown[, c("dmso_kd", "dasa_kd")])

In [34]:
cat("Mean knockdown results:")
mean_knockdown[order(mean_knockdown$mean_kd, decreasing=TRUE),]

Mean knockdown results:

,target_gene,dmso_kd,dasa_kd,mean_kd
,<chr>,<dbl>,<dbl>,<dbl>
1,KMT2B,91.37959,100.00000,95.68980
6,PIAS1,94.31813,93.96355,94.14084
4,GPBP1L1,89.76452,94.07374,91.91913
2,CHD2,95.61930,84.94617,90.28274
10,SLTM,77.11544,95.29284,86.20414
9,ADNP,79.28274,88.82017,84.05145
7,YEATS4,78.88568,83.76095,81.32332
3,BRD2,89.24758,59.32488,74.28623
5,HIC2,52.44662,73.87592,63.16127
